<a href="https://colab.research.google.com/github/MarinaZRocha/Mod_Ling_Robotica/blob/main/E03_ModLing_RAG_179741.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 Sistema RAG (Retrieval-Augmented Generation) para Manual técnico

## 🎯 O que é RAG?

**RAG** é uma técnica que combina **recuperação de informação** com **geração de texto** por IA. Em vez de depender apenas do conhecimento interno de um modelo de linguagem, o RAG:

1. **Busca** informações relevantes em uma base de dados (nossos documentos)
2. **Usa essas informações** como contexto para gerar respostas mais precisas e fundamentadas

## 🔄 Pipeline do nosso Sistema RAG

Nosso sistema funciona em 7 etapas principais:

```
📖 Documento → ✂️ Chunking → 🧮 Embeddings → 🔍 Busca → 🎯 Re-ranking → 📝 Resposta
```

1. **Carregamento**: Lemos o documento de texto
2. **Chunking**: Dividimos em pedaços menores e gerenciáveis
3. **Embeddings**: Convertemos cada pedaço em vetores numéricos que capturam o significado
4. **Expansão de Consulta**: Criamos variações da pergunta para melhorar a busca
5. **Busca**: Encontramos os pedaços mais relevantes usando similaridade vetorial
6. **Re-ranking**: Refinamos os resultados com um modelo mais preciso
7. **Geração**: Usamos os melhores trechos como contexto para gerar a resposta final

## 📖 Dataset: Datasheet ESP32-S3

Será o utilizado como dataset o Manual técnico do ESP32-S3.

---


## 🛠️ Instalação de Dependências

Primeiro, vamos instalar todas as bibliotecas que precisamos:

- **torch**: Para operações com tensores e deep learning
- **sentence-transformers**: Para criar embeddings semânticos
- **transformers**: Para tokenização e modelos de linguagem
- **numpy**: Para operações numéricas
- **requests**: Para fazer chamadas à API
- **gdown**: Para baixar arquivos do Google Drive

In [1]:
# Instalação das dependências
!pip install torch sentence-transformers transformers numpy requests gdown -q
!pip install -q pdfplumber pypdf

print("✅ Todas as dependências foram instaladas com sucesso!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 65.6 MB/s eta 0:00:00
✅ Todas as dependências foram instaladas com sucesso!


## 📥 Download dos Dados - Literatura Brasileira

Agora vamos baixar nossa base de conhecimento: uma coleção de clássicos da literatura brasileira.

Este arquivo contém obras como:
- Dom Casmurro (Machado de Assis)
- O Cortiço (Aluísio Azevedo)
- E outros clássicos brasileiros

In [14]:
import os
import requests

url = "https://raw.githubusercontent.com/MarinaZRocha/Mod_Ling_Robotica/main/esp32-s3_technical_reference_manual_en.pdf"
pdf_path = os.path.abspath("esp32-s3_technical_reference_manual_en.pdf")

print("📥 Downloading PDF from GitHub...")

response = requests.get(url, stream=True, timeout=30)
response.raise_for_status()

with open(pdf_path, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        if chunk:
            f.write(chunk)

# ✅ Validate PDF header
with open(pdf_path, "rb") as f:
    header = f.read(5)

if header != b"%PDF-":
    raise RuntimeError("❌ Downloaded file is not a valid PDF")

print("✅ PDF ready:", pdf_path)


📥 Downloading PDF from GitHub...
✅ PDF ready: /content/esp32-s3_technical_reference_manual_en.pdf


## ⚙️ Configuração Inicial

### 🔑 API Keys

Para funcionar completamente, nosso sistema precisa de:
- **OpenRouter API Key**: Para acessar modelos de linguagem avançados
- **Hugging Face Token**: Para baixar modelos (alguns são privados)

⚠️ **Importante**: Substitua as chaves abaixo pelas suas próprias chaves reais!

In [27]:
# 🔑 Configuração das chaves de API
# ⚠️ IMPORTANTE: Substitua pelas suas chaves reais!

# Você pode definir as chaves aqui diretamente ou usar variáveis de ambiente
# OPENROUTER_API_KEY = "google/gemma-3n-e2b-it:free"  # Obtenha em https://openrouter.ai/
# HUGGINGFACE_TOKEN = "seu_huggingface_token_aqui"    # Obtenha em https://huggingface.co/

# Alternativa: usar variáveis de ambiente (mais seguro)
from google.colab import userdata
OPENROUTER_API_KEY = userdata.get('UFN')
HUGGINGFACE_TOKEN = userdata.get('HF_TOKEN')

print("✅ Chaves de API configuradas!")
print(f"🔸 OpenRouter: {'✅ Configurada' if OPENROUTER_API_KEY and OPENROUTER_API_KEY != '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' else '❌ Não configurada'}")
print(f"🔸 Hugging Face: {'✅ Configurada' if HUGGINGFACE_TOKEN and HUGGINGFACE_TOKEN != 'seu_huggingface_token_aqui' else '❌ Não configurada'}")

✅ Chaves de API configuradas!
🔸 OpenRouter: ✅ Configurada
🔸 Hugging Face: ✅ Configurada


In [15]:
with open(pdf_path, "rb") as f:
    header = f.read(5)
    print("🔎 PDF header:", header)

if header != b"%PDF-":
    raise RuntimeError("❌ File is not a valid PDF")

🔎 PDF header: b'%PDF-'


In [ ]:
head esp32-s3_technical_reference_manual_en.pdf


### 🧭 Parâmetros do Sistema

Vamos definir os parâmetros que controlam como nosso sistema RAG funciona:

- **CHUNK_SIZE**: Tamanho de cada pedaço de texto (em tokens)
- **CHUNK_OVERLAP_PERCENT**: Sobreposição entre pedaços para não perder contexto
- **TOP_K_RETRIEVAL**: Quantos pedaços buscar inicialmente
- **TOP_K_RERANK**: Quantos pedaços manter após o re-ranking
- **CROSS_ENCODER_THRESHOLD**: Limiar de relevância mínima

In [16]:
# 🎛️ Parâmetros do Sistema RAG

# 📄 Configurações de Chunking (divisão do texto)
CHUNK_SIZE = 400  # tokens por chunk (aproximadamente 300 palavras)
CHUNK_OVERLAP_PERCENT = 20  # 20% de sobreposição entre chunks

# 🔍 Configurações de Busca
TOP_K_RETRIEVAL = 80  # Quantos chunks recuperar inicialmente
TOP_K_RERANK = 16     # Quantos chunks manter após re-ranking
CROSS_ENCODER_THRESHOLD = 0.2  # Limiar mínimo de relevância

# 🤖 Modelos utilizados
EMBEDDING_MODEL = 'intfloat/e5-large-v2'  # Modelo para criar embeddings
CROSS_ENCODER_MODEL = 'cross-encoder/ms-marco-MiniLM-L-6-v2'  # Para re-ranking
LLM_GENERATION_MODEL = 'openai/gpt-4o'  # Modelo para geração de texto

# 📖 Arquivo de exemplo
FILE_PATH = pdf_path

print("⚙️ Configurações do Sistema RAG:")
print(f"📏 Tamanho do chunk: {CHUNK_SIZE} tokens")
print(f"🔄 Sobreposição: {CHUNK_OVERLAP_PERCENT}%")
print(f"🔍 Busca inicial: {TOP_K_RETRIEVAL} chunks")
print(f"🎯 Re-ranking final: {TOP_K_RERANK} chunks")
print(f"📚 Dataset: Manual técnico ESP32-S3")

⚙️ Configurações do Sistema RAG:
📏 Tamanho do chunk: 400 tokens
🔄 Sobreposição: 20%
🔍 Busca inicial: 80 chunks
🎯 Re-ranking final: 16 chunks
📚 Dataset: Manual técnico ESP32-S3


## 📚 Imports e Inicialização

Agora vamos importar todas as bibliotecas que precisamos para implementar nosso sistema RAG.

In [5]:
# 📦 Imports necessários
import os
import re
import requests
import numpy as np
import torch
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer, CrossEncoder
import time
from typing import List, Tuple
from pprint import *
import pdfplumber
import pypdf


pprint("📦 Todas as bibliotecas importadas com sucesso!")
pprint(f"🖥️ Device disponível: {'GPU (CUDA)' if torch.cuda.is_available() else 'CPU'}")

'📦 Todas as bibliotecas importadas com sucesso!'
'🖥️ Device disponível: GPU (CUDA)'


## 📖 Etapa 1: Carregamento do Documento

A primeira etapa é carregar nosso documento de texto. Vamos ler "Dom Casmurro" de Machado de Assis e ver algumas estatísticas básicas sobre o texto.

**Por que essa etapa é importante?**
- Precisamos ter o texto completo em memória para processá-lo
- É importante verificar a codificação do arquivo (UTF-8, Latin-1, etc.)
- Queremos entender o tamanho e características do documento

In [17]:
def carregar_documento(caminho_arquivo: str) -> str:
    """
    Carrega o conteúdo de um arquivo PDF, tentando pdfplumber e, em caso de falha,
    tentando pypdf como alternativa.
    """
    pprint(f"📖 Carregando documento: {os.path.basename(caminho_arquivo)}")

    full_text = ""

    try:
        # Tentar com pdfplumber
        with pdfplumber.open(caminho_arquivo) as pdf:
            pprint(f"📄 Número de páginas (pdfplumber): {len(pdf.pages)}")
            for i, page in enumerate(pdf.pages):
                text = page.extract_text()
                if text:
                    full_text += text + "\n"
                else:
                    pprint(f"⚠️ Página {i+1} sem texto extraível (pdfplumber)")
        pprint("✅ Documento carregado com sucesso usando pdfplumber.")

    except Exception as e:
        pprint(f"❌ pdfplumber falhou: {e}. Tentando com pypdf...")
        full_text = ""
        try:
            # Fallback para pypdf
            with open(caminho_arquivo, 'rb') as f:
                reader = pypdf.PdfReader(f)
                num_pages = len(reader.pages)
                pprint(f"📄 Número de páginas (pypdf): {num_pages}")
                for i in range(num_pages):
                    page = reader.pages[i]
                    text = page.extract_text()
                    if text:
                        full_text += text + "\n"
                    else:
                        pprint(f"⚠️ Página {i+1} sem texto extraível (pypdf)")
            pprint("✅ Documento carregado com sucesso usando pypdf.")

        except Exception as pypdf_e:
            raise Exception(f"❌ Falha ao carregar documento com pdfplumber e pypdf: {pypdf_e}")

    if not full_text.strip():
        raise Exception("❌ O documento foi carregado, mas nenhum texto pôde ser extraído.")

    return full_text

# Carregar o documento
try:
    texto_completo = carregar_documento(FILE_PATH)

    # Estatísticas do documento
    num_caracteres = len(texto_completo)
    num_palavras = len(texto_completo.split())
    num_linhas = len(texto_completo.split('\n'))

    pprint(f"\n📊 Estatísticas do Documento:")
    pprint(f"📏 Caracteres: {num_caracteres:,}")
    pprint(f"📝 Palavras: {num_palavras:,}")
    pprint(f"📄 Linhas: {num_linhas:,}")

    # Mostrar uma prévia do início do texto
    pprint(f"\n📖 Prévia do início do documento:")
    pprint("-" * 50)
    pprint(texto_completo[:1000])  # primeiros 1000 chars
    pprint("-" * 50)

except Exception as e:
    pprint(f"❌ Erro ao carregar documento: {e}")
    pprint("\n💡 Verifique se o arquivo existe e é um PDF válido.")

'📖 Carregando documento: esp32-s3_technical_reference_manual_en.pdf'
'📄 Número de páginas (pdfplumber): 1530'
'✅ Documento carregado com sucesso usando pdfplumber.'
'\n📊 Estatísticas do Documento:'
'📏 Caracteres: 2,154,482'
'📝 Palavras: 237,101'
'📄 Linhas: 72,812'
'\n📖 Prévia do início do documento:'
'--------------------------------------------------'
('ESP32-S3\n'
 'Technical Reference Manual\n'
 'Version 1.7\n'
 'www.espressif.com\n'
 'About This Document\n'
 'TheESP32-S3TechnicalReferenceManualistargetedatdevelopersworkingonlowlevelsoftwareprojects\n'
 'thatusetheESP32-S3SoC.ItdescribesthehardwaremoduleslistedbelowfortheESP32-S3SoCandother\n'
 'productsinESP32-S3series. '
 'Themodulesdetailedinthisdocumentprovideanoverview,listoffeatures,\n'
 'hardwarearchitecturedetails,anynecessaryprogrammingprocedures,aswellasregisterdescriptions.\n'
 'Navigation in This Document\n'
 'Herearesometipsonnavigationthroughthisextensivedocument:\n'
 '• '
 'ReleaseStatusataGlanceontheverynextpageisami

In [18]:
import os

if os.path.exists(FILE_PATH):
    print(f"Verificando tipo do arquivo: {FILE_PATH}")
    !file "{FILE_PATH}"
else:
    print(f"Arquivo {FILE_PATH} não encontrado.")

Verificando tipo do arquivo: /content/esp32-s3_technical_reference_manual_en.pdf
/content/esp32-s3_technical_reference_manual_en.pdf: PDF document, version 1.5 (zip deflate encoded)


After checking the file type, if it's not a PDF, there might be an issue with the download URL or the file itself. If it is a PDF and `pdfplumber` still can't open it, we might need to try a different PDF parsing library or investigate the PDF's internal structure.

## ✂️ Etapa 2: Chunking (Divisão em Pedaços)

**Por que dividir o texto em chunks?**

1. **Limitações dos modelos**: Modelos têm limite de tokens que podem processar
2. **Precisão da busca**: Pedaços menores permitem busca mais precisa
3. **Contexto relevante**: Evitamos incluir informações desnecessárias na resposta

**Como funciona o chunking com sobreposição?**

```
Texto original: "ABCDEFGHIJKLMNOP"
Chunk 1: "ABCDEF"     (tokens 0-5)
Chunk 2: "EFGHIJ"     (tokens 4-9)  ← 2 tokens de sobreposição
Chunk 3: "IJKLMN"     (tokens 8-13) ← 2 tokens de sobreposição
```

A sobreposição garante que não perdemos contexto nas "fronteiras" entre chunks.

In [19]:
def dividir_em_chunks(texto: str, tokenizer, tamanho_chunk: int, sobreposicao_percent: int) -> List[str]:
    """
    Divide o texto em chunks baseados em contagem de tokens.

    Args:
        texto: O conteúdo do documento
        tokenizer: Tokenizer para contar tokens
        tamanho_chunk: Número máximo de tokens por chunk
        sobreposicao_percent: Percentual de sobreposição entre chunks

    Returns:
        Lista de strings, cada uma sendo um chunk
    """
    pprint(f"✂️ Dividindo texto em chunks de {tamanho_chunk} tokens...")

    # Tokenizar o texto completo
    tokens = tokenizer.encode(texto)
    total_tokens = len(tokens)
    pprint(f"📏 Total de tokens no documento: {total_tokens:,}")

    # Calcular a sobreposição
    sobreposicao_tokens = int(tamanho_chunk * (sobreposicao_percent / 100))
    passo = tamanho_chunk - sobreposicao_tokens

    pprint(f"🔄 Tokens de sobreposição: {sobreposicao_tokens}")
    pprint(f"👣 Passo entre chunks: {passo}")

    chunks_de_texto = []

    # Dividir em chunks
    for i in range(0, len(tokens), passo):
        chunk_tokens = tokens[i:i + tamanho_chunk]
        if not chunk_tokens:
            continue

        # Decodificar de volta para texto
        chunk_texto = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
        chunks_de_texto.append(chunk_texto)

    pprint(f"✅ Documento dividido em {len(chunks_de_texto)} chunks")

    return chunks_de_texto

# Inicializar o tokenizer
pprint("🔤 Inicializando tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL)

# Dividir o texto em chunks
chunks = dividir_em_chunks(texto_completo, tokenizer, CHUNK_SIZE, CHUNK_OVERLAP_PERCENT)

# Mostrar estatísticas dos chunks
pprint(f"\n📊 Estatísticas dos Chunks:")
pprint(f"📦 Número total de chunks: {len(chunks)}")

# Análise do tamanho dos chunks
tamanhos = [len(tokenizer.encode(chunk)) for chunk in chunks[:10]]  # Analisa os primeiros 10
pprint(f"📏 Tamanho médio dos primeiros 10 chunks: {np.mean(tamanhos):.1f} tokens")

# Mostrar alguns chunks de exemplo
pprint(f"\n📖 Exemplos de Chunks:")
for i in [0, len(chunks)//2, len(chunks)-1]:  # Primeiro, meio e último
    pprint(f"\n--- Chunk {i+1} ({len(tokenizer.encode(chunks[i]))} tokens) ---")
    pprint(chunks[i][:200] + "..." if len(chunks[i]) > 200 else chunks[i])

'🔤 Inicializando tokenizer...'


tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

'✂️ Dividindo texto em chunks de 400 tokens...'


Token indices sequence length is longer than the specified maximum sequence length for this model (908714 > 512). Running this sequence through the model will result in indexing errors


'📏 Total de tokens no documento: 908,714'
'🔄 Tokens de sobreposição: 80'
'👣 Passo entre chunks: 320'
'✅ Documento dividido em 2840 chunks'
'\n📊 Estatísticas dos Chunks:'
'📦 Número total de chunks: 2840'
'📏 Tamanho médio dos primeiros 10 chunks: 402.7 tokens'
'\n📖 Exemplos de Chunks:'
'\n--- Chunk 1 (401 tokens) ---'
('esp32 - s3 technical reference manual version 1. 7 www. espressif. com about '
 'this document theesp32 - '
 's3technicalreferencemanualistargetedatdevelopersworkingonlowlevelsoftwareprojects '
 'thatusetheesp32 ...')
'\n--- Chunk 1421 (404 tokens) ---'
('##world ( world1 ), thusprotectingresourcefromunauthorizedaccess ( '
 'readorwrite ), andfrom maliciousattackssuchasmalware, hardware - '
 'basedmonitoring, hardware - levelintervention, andsoon. cpus canswit...')
'\n--- Chunk 2840 (238 tokens) ---'
('##rovidedtothisdocumentforitsmerchantability, non - infringement, '
 'fitnessforanyparticular purpose, '
 'nordoesanywarrantyotherwisearisingoutofanyproposal, specification

## 🧮 Etapa 3: Vetorização (Embeddings)

**O que são embeddings?**

Embeddings são representações numéricas (vetores) que capturam o **significado semântico** do texto. Textos com significados similares têm embeddings similares.

**Exemplo conceitual:**
```
"gato" → [0.2, -0.1, 0.8, 0.3, ...] (768 dimensões)
"felino" → [0.3, -0.2, 0.7, 0.4, ...] (muito similar ao "gato")
"carro" → [-0.5, 0.9, -0.2, 0.1, ...] (bem diferente)
```

**Por que usar o modelo E5?**
- Treinado especificamente para tarefas de busca
- Suporta prefixos "query:" e "passage:" para otimizar a busca
- Boa performance em português

In [20]:
# 🧮 Inicializando o modelo de embeddings
pprint("🚀 Carregando modelo de embeddings E5-large-v2...")
pprint("⏳ Isso pode levar alguns minutos na primeira execução...")

# Carregar o modelo de embeddings
embedding_model = SentenceTransformer(EMBEDDING_MODEL)

# Verificar se GPU está disponível
device = 'cuda' if torch.cuda.is_available() else 'cpu'
pprint(f"🖥️ Modelo carregado no dispositivo: {device}")

# Mover modelo para GPU se disponível
if device == 'cuda':
    embedding_model = embedding_model.to(device)

pprint("✅ Modelo de embeddings pronto!")

'🚀 Carregando modelo de embeddings E5-large-v2...'
'⏳ Isso pode levar alguns minutos na primeira execução...'


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

'🖥️ Modelo carregado no dispositivo: cuda'
'✅ Modelo de embeddings pronto!'


In [21]:
# 🎯 Vetorizando todos os chunks do documento
pprint(f"🧮 Convertendo {len(chunks)} chunks em vetores numéricos...")
pprint("⏳ Este processo pode levar alguns minutos...")

# Adicionar o prefixo "passage:" para otimizar a busca com E5
passages_prefixed = [f"passage: {chunk}" for chunk in chunks]

# Criar embeddings para todos os chunks
start_time = time.time()
passage_embeddings = embedding_model.encode(
    passages_prefixed,
    convert_to_tensor=True,
    show_progress_bar=True,
    batch_size=32  # Processar em lotes para eficiência
)

# Mover para o dispositivo apropriado
passage_embeddings = passage_embeddings.to(device)
end_time = time.time()

pprint(f"✅ Vetorização concluída em {end_time - start_time:.1f} segundos")
pprint(f"📊 Dimensões dos embeddings: {passage_embeddings.shape}")
pprint(f"💾 Tamanho em memória: ~{passage_embeddings.numel() * 4 / 1024 / 1024:.1f} MB")

# Exemplo: mostrar a similaridade entre alguns chunks
pprint(f"\n🔍 Testando similaridade entre chunks:")
if len(chunks) >= 3:
    # Calcular similaridade entre os primeiros 3 chunks
    similarities = torch.mm(passage_embeddings[:3], passage_embeddings[:3].T)
    pprint("Matriz de similaridade (primeiros 3 chunks):")
    for i in range(3):
        for j in range(3):
            print(f"{similarities[i,j]:.3f}", end="  ")
        print()

'🧮 Convertendo 2840 chunks em vetores numéricos...'
'⏳ Este processo pode levar alguns minutos...'


Batches:   0%|          | 0/89 [00:00<?, ?it/s]

'✅ Vetorização concluída em 216.4 segundos'
'📊 Dimensões dos embeddings: torch.Size([2840, 1024])'
'💾 Tamanho em memória: ~11.1 MB'
'\n🔍 Testando similaridade entre chunks:'
'Matriz de similaridade (primeiros 3 chunks):'
1.000  0.886  0.838  
0.886  1.000  0.888  
0.838  0.888  1.000  


## 🚀 Etapa 4: Função para Chamadas à API do LLM

**Por que precisamos de um LLM externo?**

Vamos usar modelos de linguagem via API para duas tarefas importantes:
1. **Expansão de consulta**: Gerar variações da pergunta
2. **Geração da resposta final**: Criar uma resposta baseada no contexto recuperado

**OpenRouter**: É um serviço que nos dá acesso a vários modelos (GPT-4, Claude, etc.) através de uma única API.

In [28]:
def call_llm_api(prompt: str, system_prompt: str, model: str, max_retries: int = 3) -> str:
    """
    Faz uma chamada para a API do OpenRouter para obter resposta de um LLM.

    Args:
        prompt: O prompt do usuário
        system_prompt: Instrução de sistema para guiar o modelo
        model: Nome do modelo no OpenRouter
        max_retries: Número máximo de tentativas

    Returns:
        Resposta do modelo como string
    """

    for attempt in range(max_retries):
        try:
            response = requests.post(
                url="https://openrouter.ai/api/v1/chat/completions",
                headers={
                    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
                    "Content-Type": "application/json"
                },
                json={
                    "model": model,
                    "messages": [
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": prompt}
                    ],
                    "temperature": 0.7,
                    "max_tokens": 500
                }
            )

            response.raise_for_status()
            result = response.json()['choices'][0]['message']['content']
            return result.strip()

        except requests.exceptions.RequestException as e:
            pprint(f"⚠️ Tentativa {attempt + 1} falhou: {e}")
            if hasattr(e, 'response') and e.response is not None:
                pprint(f"   Status: {e.response.status_code}")
                pprint(f"   Resposta: {e.response.text[:200]}")

            if attempt == max_retries - 1:
                pprint(f"❌ Todas as {max_retries} tentativas falharam")
                return ""

            time.sleep(2 ** attempt)  # Backoff exponencial

    return ""

# Testar a conexão com a API (se as chaves estiverem configuradas)
if OPENROUTER_API_KEY and OPENROUTER_API_KEY != "sua_openrouter_api_key_aqui":
    pprint("🧪 Testando conexão com API...")
    test_response = call_llm_api(
        "Diga apenas 'Conexão funcionando'",
        "Você é um assistente útil.",
        "openai/gpt-3.5-turbo"
    )
    if test_response:
        pprint(f"✅ API funcionando: {test_response}")
    else:
        pprint("❌ Problema na conexão com API")
else:
    pprint("⚠️ Chave da API não configurada - funções de LLM não funcionarão")
    pprint("💡 Configure OPENROUTER_API_KEY para usar recursos completos")

'🧪 Testando conexão com API...'
('⚠️ Tentativa 1 falhou: 401 Client Error: Unauthorized for url: '
 'https://openrouter.ai/api/v1/chat/completions')
'   Status: 401'
('   Resposta: {"error":{"message":"No cookie auth credentials '
 'found","code":401}}')
('⚠️ Tentativa 2 falhou: 401 Client Error: Unauthorized for url: '
 'https://openrouter.ai/api/v1/chat/completions')
'   Status: 401'
('   Resposta: {"error":{"message":"No cookie auth credentials '
 'found","code":401}}')
('⚠️ Tentativa 3 falhou: 401 Client Error: Unauthorized for url: '
 'https://openrouter.ai/api/v1/chat/completions')
'   Status: 401'
('   Resposta: {"error":{"message":"No cookie auth credentials '
 'found","code":401}}')
'❌ Todas as 3 tentativas falharam'
'❌ Problema na conexão com API'


## 🔍 Etapa 5a: Expansão de Consulta - Multi-Query

**Por que expandir a consulta?**

Uma única pergunta pode ser formulada de várias maneiras. Ao criar variações, aumentamos a chance de encontrar informações relevantes.

**Exemplo:**
- Pergunta original: *"Qual a cor dos olhos de Capitu?"*
- Variações possíveis:
  - *"Como eram os olhos de Capitu?"*
  - *"Descreva os olhos da personagem Capitu"*
  - *"Que características tinham os olhos de Capitu?"*

Cada variação pode ativar chunks diferentes e complementares.

In [26]:
def gerar_expansao_multi_query(query: str) -> List[str]:
    """
    Usa um LLM para gerar múltiplas paráfrases da consulta original.

    Args:
        query: A pergunta original do usuário

    Returns:
        Lista de perguntas reformuladas
    """
    pprint("🔄 Gerando variações da pergunta (Multi-Query)...")

    system_prompt = """
Você é um assistente especializado em reformular perguntas.
Sua tarefa é gerar 4 reformulações diferentes de uma pergunta para melhorar a busca em documentos.
As reformulações devem:
- Manter o mesmo significado da pergunta original
- Usar sinônimos e estruturas diferentes
- Ser claras e diretas
- Cobrir diferentes formas de expressar a mesma ideia

Retorne apenas as 4 perguntas, uma por linha, sem numeração ou marcadores.
"""

    response = call_llm_api(query, system_prompt, "openai/gpt-3.5-turbo")

    if response:
        # Extrair as perguntas da resposta
        queries = [q.strip() for q in response.split('\n') if q.strip()]
        queries = [q for q in queries if len(q) > 5]  # Filtrar respostas muito curtas

        pprint(f"✅ {len(queries)} variações geradas:")
        for i, q in enumerate(queries, 1):
            pprint(f"   {i}. {q}")

        return queries

    pprint("❌ Não foi possível gerar variações")
    return []

# Exemplo de uso - vamos testar com uma pergunta sobre Dom Casmurro
query_exemplo = "Quais são os pinos de conexão I2C?"
pprint(f"🎯 Pergunta original: '{query_exemplo}'\n")

if OPENROUTER_API_KEY and OPENROUTER_API_KEY != "sua_openrouter_api_key_aqui":
    multi_queries_exemplo = gerar_expansao_multi_query(query_exemplo)
else:
    pprint("⚠️ Simulando variações (API não configurada):")
    multi_queries_exemplo = [
        "Quais são os GPIO para I2C?",
        "Como é descrita a conexão I2C?",
        "Quais pinos tem identificação de I2C?",
        "A conexão I2C é mencionada?"
    ]
    for i, q in enumerate(multi_queries_exemplo, 1):
        pprint(f"   {i}. {q}")

"🎯 Pergunta original: 'Quais são os pinos de conexão I2C?'\n"
'🔄 Gerando variações da pergunta (Multi-Query)...'
('⚠️ Tentativa 1 falhou: 401 Client Error: Unauthorized for url: '
 'https://openrouter.ai/api/v1/chat/completions')
'   Status: 401'
('   Resposta: {"error":{"message":"No cookie auth credentials '
 'found","code":401}}')
('⚠️ Tentativa 2 falhou: 401 Client Error: Unauthorized for url: '
 'https://openrouter.ai/api/v1/chat/completions')
'   Status: 401'
('   Resposta: {"error":{"message":"No cookie auth credentials '
 'found","code":401}}')
('⚠️ Tentativa 3 falhou: 401 Client Error: Unauthorized for url: '
 'https://openrouter.ai/api/v1/chat/completions')
'   Status: 401'
('   Resposta: {"error":{"message":"No cookie auth credentials '
 'found","code":401}}')
'❌ Todas as 3 tentativas falharam'
'❌ Não foi possível gerar variações'


## 🔍 Etapa 5b: Expansão de Consulta - HyDE

**O que é HyDE (Hypothetical Document Embeddings)?**

HyDE é uma técnica onde pedimos para um LLM **imaginar uma resposta** para nossa pergunta. Depois, usamos essa resposta hipotética como uma consulta adicional.

**Por que isso funciona?**
- A resposta hipotética geralmente tem vocabulário e estrutura similar ao texto real que procuramos
- Isso melhora a similaridade vetorial na busca

**Exemplo:**
- Pergunta: *"Os olhos de Capitu eram claros ou escuros?"*
- Resposta hipotética: *"Os olhos de Capitu eram descritos como escuros, de uma cor intensa que chamava a atenção de Bentinho..."*

Esta resposta hipotética pode ser mais similar aos trechos reais do livro do que a pergunta original.

In [ ]:
def gerar_resposta_hipotetica(query: str) -> str:
    """
    Usa um LLM para gerar uma resposta hipotética para a consulta (HyDE).

    Args:
        query: A pergunta do usuário

    Returns:
        Uma resposta hipotética que pode melhorar a busca
    """
    pprint("💭 Gerando resposta hipotética (HyDE)...")

    system_prompt = """
Você é um assistente especializado em Manuais técnicos de microcontroladores.
Gere uma resposta hipotética para a pergunta do usuário, como se fosse uma informação de um manual técnico.

A resposta deve:
- Ser plausível e bem estruturada
- Ter entre 2-4 frases
- Usar vocabulário típico de análise literária
- Não afirmar certezas absolutas (use "parece", "sugere", "indica", etc.)

Não mencione que é uma resposta hipotética.
"""

    response = call_llm_api(query, system_prompt, "openai/gpt-3.5-turbo")

    if response:
        pprint(f"✅ Resposta hipotética gerada:")
        pprint(f"   📝 '{response}'")
        return response

    pprint("❌ Não foi possível gerar resposta hipotética")
    return ""

# Exemplo de uso com a mesma pergunta
pprint(f"🎯 Pergunta: '{query_exemplo}'\n")

if OPENROUTER_API_KEY and OPENROUTER_API_KEY != "sua_openrouter_api_key_aqui":
    hyde_exemplo = gerar_resposta_hipotetica(query_exemplo)
else:
    pprint("⚠️ Simulando resposta hipotética (API não configurada):")
    hyde_exemplo = "The ESP32-S3 integrates a complex structure, including a QACC Accumulator Register ..."
    pprint(f"   📝 '{hyde_exemplo}'")

"🎯 Pergunta: 'Os olhos de Capitu eram claros ou escuros?'\n"
'💭 Gerando resposta hipotética (HyDE)...'
'✅ Resposta hipotética gerada:'
('   📝 \'A ambiguidade em torno da cor dos olhos de Capitu em "Dom Casmurro" '
 'de Machado de Assis é um dos elementos mais discutidos da obra. A narrativa '
 'sugere tanto a luminosidade de olhos claros quanto a profundidade de olhos '
 'escuros, deixando espaço para interpretações diversas por parte dos '
 "leitores.'")


## 🎯 Etapa 6: Busca Híbrida (Retrieval)

**Como funciona a busca híbrida?**

1. **Vetorizar consultas**: Convertemos todas as consultas (original + variações + HyDE) em embeddings
2. **Calcular similaridades**: Comparamos cada consulta com todos os chunks do documento
3. **Combinação inteligente**: Para cada chunk, pegamos o **maior score** entre todas as consultas
4. **Ranking**: Selecionamos os chunks com maiores scores

**Por que "híbrida"?**
- Combinamos múltiplas estratégias (multi-query + HyDE)
- Diferentes consultas podem ativar chunks complementares
- Aumentamos a robustez da busca

In [ ]:
def busca_hibrida(query_original: str, multi_queries: List[str], hyde_doc: str,
                  passage_embeddings: torch.Tensor, embedding_model,
                  top_k: int = 80) -> Tuple[List[str], List[int], List[float]]:
    """
    Realiza busca híbrida usando consulta original, variações e documento hipotético.

    Args:
        query_original: Pergunta original
        multi_queries: Lista de variações da pergunta
        hyde_doc: Documento hipotético gerado
        passage_embeddings: Embeddings dos chunks do documento
        embedding_model: Modelo para gerar embeddings das consultas
        top_k: Número de chunks a retornar

    Returns:
        Tupla com (chunks_recuperados, indices, scores)
    """
    pprint(f"🔍 Iniciando busca híbrida (top-{top_k})...")

    # Preparar todas as consultas
    all_queries = [query_original] + multi_queries
    if hyde_doc:
        all_queries.append(hyde_doc)

    pprint(f"📋 Usando {len(all_queries)} consultas para busca")

    # Adicionar prefixo "query:" para E5
    queries_prefixed = [f"query: {q}" for q in all_queries]

    # Vetorizar todas as consultas
    pprint(f"🧮 Vetorizando {len(queries_prefixed)} consultas...")
    query_embeddings = embedding_model.encode(
        queries_prefixed,
        convert_to_tensor=True,
        show_progress_bar=True
    )
    query_embeddings = query_embeddings.to(passage_embeddings.device)

    # Calcular similaridade entre todas as consultas e todos os passages
    pprint(f"⚡ Calculando similaridades...")
    similarities = torch.mm(query_embeddings, passage_embeddings.T)

    # Para cada passage, pegar o score máximo entre todas as consultas
    max_scores, best_query_idx = torch.max(similarities, dim=0)

    # Pegar os top-k resultados
    top_scores, top_indices = torch.topk(max_scores, k=min(top_k, len(chunks)))

    # Converter para listas Python
    top_indices = top_indices.cpu().tolist()
    top_scores = top_scores.cpu().tolist()

    # Recuperar os chunks correspondentes
    retrieved_chunks = [chunks[i] for i in top_indices]

    pprint(f"✅ {len(retrieved_chunks)} chunks recuperados")
    pprint(f"📊 Scores: {top_scores[0]:.3f} (melhor) → {top_scores[-1]:.3f} (pior)")

    return retrieved_chunks, top_indices, top_scores

# Demonstração da busca híbrida
pprint(f"🎯 Realizando busca para: '{query_exemplo}'\n")

# Usar as consultas que já geramos (ou simuladas)
retrieved_chunks, retrieved_indices, retrieval_scores = busca_hibrida(
    query_exemplo,
    multi_queries_exemplo,
    hyde_exemplo,
    passage_embeddings,
    embedding_model,
    TOP_K_RETRIEVAL
)

# Mostrar alguns exemplos dos chunks recuperados
pprint(f"\n📖 Exemplos dos chunks com melhor score:")
for i in range(min(3, len(retrieved_chunks))):
    chunk_idx = retrieved_indices[i]
    score = retrieval_scores[i]
    chunk_preview = retrieved_chunks[i][:300] + "..." if len(retrieved_chunks[i]) > 300 else retrieved_chunks[i]

    pprint(f"\n--- Chunk {chunk_idx} (Score: {score:.3f}) ---")
    pprint(chunk_preview)

"🎯 Realizando busca para: 'Os olhos de Capitu eram claros ou escuros?'\n"
'🔍 Iniciando busca híbrida (top-80)...'
'📋 Usando 6 consultas para busca'
'🧮 Vetorizando 6 consultas...'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'⚡ Calculando similaridades...'
'✅ 80 chunks recuperados'
'📊 Scores: 0.848 (melhor) → 0.822 (pior)'
'\n📖 Exemplos dos chunks com melhor score:'
'\n--- Chunk 107 (Score: 0.848) ---'
('##imo ; hoje mesmo ele ha de falar. - - voce jura? - - juro! deixe ver os '
 'olhos, capitu. tinha - me lembrado a definico que jose dias dera deles, " '
 'olhos de cigana obliqua e dissimulada. " eu nao sabia o que era obliqua, mas '
 'dissimulada sabia, e queria ver se podiam chamar assim. capitu deixou - se '
 '...')
'\n--- Chunk 112 (Score: 0.848) ---'
('ate que exclamei : - - pronto! - - estara bom? - - veja no espelho. em vez '
 'de ir ao espelho, que pensais que fez capitu? nao vos esquecais que estava '
 'sentada, de costas para mim. capitu derreou a cabeca, a tal ponto que me foi '
 'preciso acudir com as maos e ampara - la ; o espaldar da cadeira era baix...')
'\n--- Chunk 213 (Score: 0.846) ---'
('os olhos, aperta - los bem, esquecer tudo para dormir, mas nao dormia. esse '
 'mesmo trabalho fez 

## 🎯 Etapa 7: Re-ranking com Cross-Encoder

**Por que fazer re-ranking?**

A busca vetorial (embeddings) é **rápida** mas às vezes **imprecisa**. O Cross-Encoder é mais **lento** mas muito mais **preciso** para avaliar relevância.

**Estratégia de 2 estágios:**
1. **Busca vetorial**: Encontra 80 candidatos rapidamente
2. **Re-ranking**: Reavalia os 80 candidatos com precisão e seleciona os 16 melhores

**Como funciona o Cross-Encoder?**
- Recebe a pergunta + chunk juntos como entrada
- Analisa a relevância diretamente (não através de similaridade vetorial)
- Retorna um score de 0-1 indicando relevância

In [ ]:
# 🎯 Inicializar o modelo Cross-Encoder para re-ranking
pprint("🚀 Carregando modelo Cross-Encoder para re-ranking...")
pprint("⏳ Primeira execução pode demorar alguns minutos...")

cross_encoder = CrossEncoder(CROSS_ENCODER_MODEL)
pprint("✅ Cross-Encoder carregado!")

'🚀 Carregando modelo Cross-Encoder para re-ranking...'
'⏳ Primeira execução pode demorar alguns minutos...'


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

'✅ Cross-Encoder carregado!'


In [ ]:
def fazer_reranking(query: str, chunks_recuperados: List[str], indices_recuperados: List[int],
                   cross_encoder, top_k_final: int = 16, threshold: float = 0.2) -> Tuple[List[int], List[float]]:
    """
    Re-rankeia os chunks recuperados usando Cross-Encoder.

    Args:
        query: Pergunta original
        chunks_recuperados: Lista de chunks da busca inicial
        indices_recuperados: Índices originais dos chunks
        cross_encoder: Modelo Cross-Encoder
        top_k_final: Número final de chunks a manter
        threshold: Score mínimo para considerar relevante

    Returns:
        Tupla com (indices_finais, scores_finais)
    """
    pprint(f"🎯 Re-rankeando {len(chunks_recuperados)} chunks com Cross-Encoder...")

    # Preparar pares [query, chunk] para o Cross-Encoder
    cross_encoder_input = [[query, chunk] for chunk in chunks_recuperados]

    # Calcular scores de relevância
    pprint(f"⚡ Calculando scores de relevância...")
    cross_scores = cross_encoder.predict(cross_encoder_input, show_progress_bar=True)

    # Combinar scores com índices
    scored_results = list(zip(cross_scores, indices_recuperados))

    # Ordenar por score (decrescente)
    scored_results.sort(key=lambda x: x[0], reverse=True)

    # Aplicar threshold e pegar top-k
    filtered_results = [(score, idx) for score, idx in scored_results if score >= threshold]
    final_results = filtered_results[:top_k_final]

    if not final_results:
        pprint(f"⚠️ Nenhum chunk passou do threshold {threshold}")
        # Se nenhum passou do threshold, pegar os melhores mesmo assim
        final_results = scored_results[:min(3, len(scored_results))]
        pprint(f"📋 Usando {len(final_results)} melhores chunks mesmo abaixo do threshold")

    final_scores = [score for score, idx in final_results]
    final_indices = [idx for score, idx in final_results]

    pprint(f"✅ {len(final_results)} chunks finais selecionados")
    pprint(f"📊 Scores finais: {final_scores[0]:.3f} (melhor) → {final_scores[-1]:.3f} (pior)")

    return final_indices, final_scores

# Aplicar re-ranking aos chunks recuperados
pprint(f"🎯 Aplicando re-ranking aos chunks recuperados...\n")

final_indices, final_scores = fazer_reranking(
    query_exemplo,
    retrieved_chunks,
    retrieved_indices,
    cross_encoder,
    TOP_K_RERANK,
    CROSS_ENCODER_THRESHOLD
)

# Mostrar os chunks com melhor score após re-ranking
pprint(f"\n📖 Top chunks após re-ranking:")
for i, (chunk_idx, score) in enumerate(zip(final_indices[:3], final_scores[:3])):
    chunk_text = chunks[chunk_idx]
    chunk_preview = chunk_text[:400] + "..." if len(chunk_text) > 400 else chunk_text

    pprint(f"\n--- Posição {i+1}: Chunk {chunk_idx} (Score: {score:.3f}) ---")
    pprint(chunk_preview)

    # Highlighting: tentar encontrar menções relevantes
    keywords = ["olhos", "Capitu", "olhar", "vista", "cor"]
    found_keywords = [kw for kw in keywords if kw.lower() in chunk_text.lower()]
    if found_keywords:
        pprint(f"🔑 Palavras-chave encontradas: {', '.join(found_keywords)}")

'🎯 Aplicando re-ranking aos chunks recuperados...\n'
'🎯 Re-rankeando 80 chunks com Cross-Encoder...'
'⚡ Calculando scores de relevância...'


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

'✅ 16 chunks finais selecionados'
'📊 Scores finais: 5.049 (melhor) → 3.167 (pior)'
'\n📖 Top chunks após re-ranking:'
'\n--- Posição 1: Chunk 46 (Score: 5.049) ---'
('##iatura serafica. os olhos continuaram a dizer coisas infinitas, as '
 'palavras de boca e que nem tentavam sair, tornavam ao coraco caladas como '
 'vinham... capitulo xv outra voz repentina outra voz repentina, mas desta vez '
 'uma voz de homem : - - voces estao jogando o siso? era o pai de capitu, que '
 'estava a porta dos fundos, ao pe da mulher. soltamos as maos depressa, e '
 'ficamos atrapalhados. capitu fo...')
'🔑 Palavras-chave encontradas: olhos, Capitu, cor'
'\n--- Posição 2: Chunk 112 (Score: 4.759) ---'
('ate que exclamei : - - pronto! - - estara bom? - - veja no espelho. em vez '
 'de ir ao espelho, que pensais que fez capitu? nao vos esquecais que estava '
 'sentada, de costas para mim. capitu derreou a cabeca, a tal ponto que me foi '
 'preciso acudir com as maos e ampara - la ; o espaldar da cadeira era

## 📝 Etapa 8: Geração da Resposta Final

**Última etapa: sintetizar a resposta**

Agora que temos os chunks mais relevantes, vamos:
1. **Combinar** os chunks em um contexto estruturado
2. **Enviar** para um LLM junto com a pergunta original
3. **Gerar** uma resposta fundamentada apenas no contexto fornecido

**Prompting estratégico:**
- Instruímos o modelo a citar as fontes
- Pedimos para usar apenas informações do contexto
- Solicitamos resposta concisa e direta

In [ ]:
def gerar_resposta_final(query: str, chunk_indices: List[int], chunk_scores: List[float],
                        chunks: List[str], call_llm_api, model: str) -> str:
    """
    Gera a resposta final usando os chunks mais relevantes como contexto.

    Args:
        query: Pergunta original
        chunk_indices: Índices dos chunks selecionados
        chunk_scores: Scores de relevância dos chunks
        chunks: Lista completa de chunks
        call_llm_api: Função para chamar API do LLM
        model: Modelo a usar para geração

    Returns:
        Resposta final gerada
    """
    pprint(f"📝 Gerando resposta final usando {len(chunk_indices)} chunks como contexto...")

    # Construir o contexto com os chunks selecionados
    context_parts = []
    for i, (chunk_idx, score) in enumerate(zip(chunk_indices, chunk_scores)):
        context_parts.append(f"Fonte {i+1} (Chunk {chunk_idx}, Relevância: {score:.2f}):")
        context_parts.append(f'"""\n{chunks[chunk_idx]}\n"""')
        context_parts.append("")  # Linha em branco

    context = "\n".join(context_parts)

    # System prompt para guiar a geração
    system_prompt = """
Você é um assistente especializado em análise literária.

Sua tarefa é responder à pergunta do usuário baseando-se EXCLUSIVAMENTE nos trechos
de texto fornecidos como contexto.

Instruções importantes:
1. Use APENAS as informações presentes no contexto fornecido
2. Se a informação estiver explícita, cite a fonte: [Fonte X]
3. Se precisar sintetizar informações de múltiplas fontes, cite todas: [Fontes X, Y]
4. Se não encontrar informação suficiente, diga que não foi possível responder com base no contexto
5. Seja conciso e direto
6. Não adicione conhecimento externo ao documento

Responda de forma clara e bem fundamentada.
"""

    # Prompt final
    final_prompt = f"""
CONTEXTO:
{context}

PERGUNTA: {query}

RESPOSTA:
"""

    pprint(f"🤖 Enviando para LLM (contexto: {len(context)} caracteres)...")

    # Gerar resposta
    response = call_llm_api(final_prompt, system_prompt, model)

    if response:
        pprint(f"✅ Resposta gerada com sucesso!")
        return response
    else:
        return "❌ Não foi possível gerar uma resposta. Verifique a configuração da API."

# Gerar a resposta final
pprint(f"🎯 Pergunta: '{query_exemplo}'\n")

if OPENROUTER_API_KEY and OPENROUTER_API_KEY != "sua_openrouter_api_key_aqui":
    resposta_final = gerar_resposta_final(
        query_exemplo,
        final_indices,
        final_scores,
        chunks,
        call_llm_api,
        LLM_GENERATION_MODEL
    )
else:
    pprint("⚠️ Simulando resposta (API não configurada)...")
    resposta_final = "GPIO43 and GPIO44 are the pins available for I2C connections. [Fontes: matual técnico]"

pprint("\n" + "="*60)
pprint("🎉 RESPOSTA FINAL DO SISTEMA RAG")
pprint("="*60)
pprint(f"❓ Pergunta: {query_exemplo}")
pprint(f"\n💡 Resposta: {resposta_final}")
pprint("="*60)

"🎯 Pergunta: 'Os olhos de Capitu eram claros ou escuros?'\n"
'📝 Gerando resposta final usando 16 chunks como contexto...'
'🤖 Enviando para LLM (contexto: 18000 caracteres)...'
'✅ Resposta gerada com sucesso!'
'\n============================================================'
'🎉 RESPOSTA FINAL DO SISTEMA RAG'
'============================================================'
'❓ Pergunta: Os olhos de Capitu eram claros ou escuros?'
('\n'
 '💡 Resposta: Não foi possível determinar se os olhos de Capitu eram claros ou '
 'escuros com base no contexto fornecido. As fontes mencionam características '
 'dos olhos de Capitu, como sendo "de ressaca" e "olhos de cigana obliqua e '
 'dissimulada" [Fontes 5, 8], mas não especificam claramente a cor dos olhos.')
'============================================================'


## 📖 Análise dos Chunks Utilizados

Vamos examinar em detalhes os trechos que foram utilizados para gerar a resposta. Isso nos ajuda a entender:
- **Quais partes** do documento foram consideradas mais relevantes
- **Por que** o sistema escolheu esses trechos específicos
- **Como** a resposta foi construída a partir do contexto

In [ ]:
# 📖 Análise detalhada dos chunks utilizados
pprint("📚 ANÁLISE DOS TRECHOS UTILIZADOS")
pprint("="*50)

for i, (chunk_idx, score) in enumerate(zip(final_indices, final_scores)):
    chunk_text = chunks[chunk_idx]

    pprint(f"\n📄 FONTE {i+1} - Chunk {chunk_idx}")
    pprint(f"🎯 Score de Relevância: {score:.3f}")
    pprint(f"📏 Tamanho: {len(chunk_text)} caracteres")
    pprint("-" * 40)
    pprint(chunk_text)
    pprint("-" * 40)

    # Análise de palavras-chave
    keywords_analise = ["pins", "GPIO", "I2C", "connection", "identification", "ID", "description"]
    palavras_encontradas = []

    for keyword in keywords_analise:
        count = chunk_text.lower().count(keyword.lower())
        if count > 0:
            palavras_encontradas.append(f"{keyword} ({count}x)")

    if palavras_encontradas:
        pprint(f"🔍 Palavras-chave encontradas: {', '.join(palavras_encontradas)}")

    pprint("\n" + "="*50)

pprint(f"\n📊 RESUMO DA ANÁLISE:")
pprint(f"📋 Total de chunks analisados no re-ranking: {len(retrieved_chunks)}")
pprint(f"🎯 Chunks finais selecionados: {len(final_indices)}")
pprint(f"📈 Score mais alto: {max(final_scores):.3f}")
pprint(f"📉 Score mais baixo: {min(final_scores):.3f}")
pprint(f"⚖️ Score médio: {np.mean(final_scores):.3f}")


'📚 ANÁLISE DOS TRECHOS UTILIZADOS'
'=================================================='
'\n📄 FONTE 1 - Chunk 46'
'🎯 Score de Relevância: 5.049'
'📏 Tamanho: 1049 caracteres'
'----------------------------------------'
('##iatura serafica. os olhos continuaram a dizer coisas infinitas, as '
 'palavras de boca e que nem tentavam sair, tornavam ao coraco caladas como '
 'vinham... capitulo xv outra voz repentina outra voz repentina, mas desta vez '
 'uma voz de homem : - - voces estao jogando o siso? era o pai de capitu, que '
 'estava a porta dos fundos, ao pe da mulher. soltamos as maos depressa, e '
 'ficamos atrapalhados. capitu foi ao muro, e, com o prego, disfarcadamente, '
 'apagou os nossos nomes escritos. - - capitu! - - papai! - - nao me estragues '
 'o reboco do muro. capitu riscava sobre o riscado, para apagar bem o escrito. '
 'padua saiu ao quintal, a ver o que era, mas ja a filha tinha comecado outra '
 'coisa, um perfil, que disse ser o retrato dele, e tanto podia ser dele c

## 🔧 Sistema RAG Completo - Função Interativa

Agora vamos criar uma função que integra todo o pipeline RAG, permitindo fazer perguntas interativas sobre o documento.

Esta função executa todo o processo:
1. ✂️ Expansão da consulta
2. 🔍 Busca híbrida
3. 🎯 Re-ranking
4. 📝 Geração da resposta

In [ ]:
def sistema_rag_completo(pergunta: str, verbose: bool = True) -> dict:
    """
    Sistema RAG completo que processa uma pergunta e retorna uma resposta fundamentada.

    Args:
        pergunta: A pergunta do usuário
        verbose: Se deve imprimir informações detalhadas do processo

    Returns:
        Dicionário com a resposta e metadados do processo
    """
    if verbose:
        pprint(f"🚀 Iniciando Sistema RAG para: '{pergunta}'")
        pprint("="*60)

    resultado = {
        "pergunta": pergunta,
        "resposta": "",
        "chunks_utilizados": [],
        "scores": [],
        "sucesso": False
    }

    try:
        # Etapa 1: Expansão da consulta
        if verbose:
            pprint("\n🔄 Etapa 1: Expansão da consulta")

        if OPENROUTER_API_KEY and OPENROUTER_API_KEY != "sua_openrouter_api_key_aqui":
            multi_queries = gerar_expansao_multi_query(pergunta)
            hyde_doc = gerar_resposta_hipotetica(pergunta)
        else:
            if verbose:
                pprint("⚠️ API não configurada - usando consulta original apenas")
            multi_queries = []
            hyde_doc = ""

        # Etapa 2: Busca híbrida
        if verbose:
            pprint("\n🔍 Etapa 2: Busca híbrida")

        retrieved_chunks_local, retrieved_indices_local, retrieval_scores_local = busca_hibrida(
            pergunta, multi_queries, hyde_doc, passage_embeddings, embedding_model, TOP_K_RETRIEVAL
        )

        # Etapa 3: Re-ranking
        if verbose:
            pprint("\n🎯 Etapa 3: Re-ranking")

        final_indices_local, final_scores_local = fazer_reranking(
            pergunta, retrieved_chunks_local, retrieved_indices_local,
            cross_encoder, TOP_K_RERANK, CROSS_ENCODER_THRESHOLD
        )

        # Etapa 4: Geração da resposta
        if verbose:
            pprint("\n📝 Etapa 4: Geração da resposta")

        if not final_indices_local:
            resposta = "Desculpe, não encontrei informações relevantes para responder à sua pergunta no documento."
        else:
            if OPENROUTER_API_KEY and OPENROUTER_API_KEY != "sua_openrouter_api_key_aqui":
                resposta = gerar_resposta_final(
                    pergunta, final_indices_local, final_scores_local,
                    chunks, call_llm_api, LLM_GENERATION_MODEL
                )
            else:
                # Fallback quando API não está configurada
                resposta = f"Encontrei {len(final_indices_local)} trechos relevantes, mas preciso da API configurada para gerar uma resposta elaborada. Configure OPENROUTER_API_KEY para funcionalidade completa."

        # Preencher resultado
        resultado["resposta"] = resposta
        resultado["chunks_utilizados"] = final_indices_local
        resultado["scores"] = final_scores_local
        resultado["sucesso"] = True

        if verbose:
            pprint("\n" + "="*60)
            pprint("✅ PROCESSO CONCLUÍDO COM SUCESSO!")
            pprint("="*60)
            pprint(f"❓ Pergunta: {pergunta}")
            pprint(f"\n💡 Resposta: {resposta}")
            pprint("="*60)

    except Exception as e:
        if verbose:
            print(f"❌ Erro durante o processamento: {e}")
        resultado["resposta"] = f"Erro: {e}"

    return resultado

pprint("🎉 Sistema RAG completo definido!")
pprint("\n💡 Agora você pode fazer perguntas usando: sistema_rag_completo('sua pergunta aqui')")

'🎉 Sistema RAG completo definido!'
('\n'
 "💡 Agora você pode fazer perguntas usando: sistema_rag_completo('sua pergunta "
 "aqui')")


## 🎮 Teste Interativo - Faça suas Perguntas!

Agora você pode testar o sistema com diferentes perguntas sobre Dom Casmurro.

**Exemplos de perguntas que você pode fazer:**
- "Qual a profissão de Bentinho?"
- "Como Capitu e Bentinho se conheceram?"
- "Quem é José Dias?"
- "O que acontece no final da história?"
- "Qual é o tema principal do livro?"

In [ ]:
# 🎮 Teste do sistema com diferentes perguntas

perguntas_teste = [
    "W",
    "Como Bentinho e Capitu se conheceram?"
]

pprint("🧪 Testando o sistema RAG com perguntas de exemplo:\n")

for i, pergunta in enumerate(perguntas_teste, 1):
    pprint(f"\n📝 TESTE {i}/4")
    pprint("="*50)

    resultado = sistema_rag_completo(pergunta, verbose=False)

    pprint(f"❓ Pergunta: {pergunta}")
    pprint(f"💡 Resposta: {resultado['resposta']}")

    if resultado['chunks_utilizados']:
        pprint(f"📊 Chunks utilizados: {len(resultado['chunks_utilizados'])}")
        pprint(f"🎯 Melhor score: {max(resultado['scores']):.3f}")

    pprint("-" * 50)

pprint("\n✅ Testes concluídos!")

'🧪 Testando o sistema RAG com perguntas de exemplo:\n'
'\n📝 TESTE 1/4'
'=================================================='
'🔄 Gerando variações da pergunta (Multi-Query)...'
'✅ 4 variações geradas:'
'   1. Qual é o papel de José Dias na obra literária?'
'   2. Qual personagem é José Dias no livro?'
'   3. Quem representa José Dias na trama?'
'   4. Qual a importância de José Dias na história?'
'💭 Gerando resposta hipotética (HyDE)...'
'✅ Resposta hipotética gerada:'
('   📝 \'José Dias, personagem de "Dom Casmurro" de Machado de Assis, parece '
 'ser um confidente astuto e perspicaz, que desempenha um papel significativo '
 'na trama ao aconselhar Bentinho e influenciar suas decisões. Sua presença '
 'parece indicar a complexidade das relações interpessoais e o jogo de '
 "interesses presentes na narrativa machadiana.'")
'🔍 Iniciando busca híbrida (top-80)...'
'📋 Usando 6 consultas para busca'
'🧮 Vetorizando 6 consultas...'


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'⚡ Calculando similaridades...'
'✅ 80 chunks recuperados'
'📊 Scores: 0.848 (melhor) → 0.806 (pior)'
'🎯 Re-rankeando 80 chunks com Cross-Encoder...'
'⚡ Calculando scores de relevância...'


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

'✅ 16 chunks finais selecionados'
'📊 Scores finais: 4.053 (melhor) → 1.479 (pior)'
'📝 Gerando resposta final usando 16 chunks como contexto...'
'🤖 Enviando para LLM (contexto: 18042 caracteres)...'
'✅ Resposta gerada com sucesso!'
'❓ Pergunta: Quem é José Dias no livro?'
('💡 Resposta: José Dias é um personagem que aparece em várias situações no '
 'livro. Ele é descrito como alguém que está próximo à família do '
 'protagonista, exercendo influência e demonstrando afeição. Em várias '
 'ocasiões, ele é mencionado em contextos ligados à família do narrador e suas '
 'decisões de vida. Por exemplo, José Dias é quem comunica ao protagonista '
 'sobre a saudade que sua mãe sente dele, pintando a tristeza dela com '
 'admiração [Fonte 4]. Ele também é responsável por discutir o futuro do '
 'narrador em relação à sua formação religiosa, demonstrando ser uma pessoa '
 'próxima e de confiança, a ponto de ser encarregado de assuntos importantes '
 'da família [Fonte 14]. Além disso, ele é vist

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'⚡ Calculando similaridades...'
'✅ 80 chunks recuperados'
'📊 Scores: 0.852 (melhor) → 0.828 (pior)'
'🎯 Re-rankeando 80 chunks com Cross-Encoder...'
'⚡ Calculando scores de relevância...'


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

'✅ 16 chunks finais selecionados'
'📊 Scores finais: 4.222 (melhor) → 2.582 (pior)'
'📝 Gerando resposta final usando 16 chunks como contexto...'
'🤖 Enviando para LLM (contexto: 17924 caracteres)...'
'✅ Resposta gerada com sucesso!'
'❓ Pergunta: Como Bentinho e Capitu se conheceram?'
'💡 Resposta: Não foi possível responder com base no contexto fornecido.'
'📊 Chunks utilizados: 16'
'🎯 Melhor score: 4.222'
'--------------------------------------------------'
'\n✅ Testes concluídos!'


In [ ]:
# 🎯 Espaço para suas próprias perguntas!
# Modifique a pergunta abaixo e execute a célula para testar

sua_pergunta = "A obra menciona alguma comida ou prato típico?"

pprint("🔍 Processando sua pergunta...\n")
resultado_personalizado = sistema_rag_completo(sua_pergunta, verbose=True)

'🔍 Processando sua pergunta...\n'
"🚀 Iniciando Sistema RAG para: 'A obra menciona alguma comida ou prato típico?'"
'============================================================'
'\n🔄 Etapa 1: Expansão da consulta'
'🔄 Gerando variações da pergunta (Multi-Query)...'
'✅ 4 variações geradas:'
'   1. - Na obra é citada alguma alimentação ou prato característico?'
'   2. - Há menção de alguma comida ou prato tradicional na obra?'
'   3. - Algum alimento ou prato típico é mencionado no texto?'
'   4. - Existe alguma referência a comida ou prato típico na obra?'
'💭 Gerando resposta hipotética (HyDE)...'
'✅ Resposta hipotética gerada:'
("   📝 'Em determinados trechos da obra, é possível identificar referências "
 'sutis a comidas que remetem à cultura brasileira. A presença de pratos '
 'típicos parece sugerir uma conexão simbólica entre a alimentação e as '
 'questões identitárias exploradas pelo autor, enriquecendo a narrativa com '
 "elementos da culinária nacional.'")
'\n🔍 Etapa 2: Busca hí

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'⚡ Calculando similaridades...'
'✅ 80 chunks recuperados'
'📊 Scores: 0.829 (melhor) → 0.809 (pior)'
'\n🎯 Etapa 3: Re-ranking'
'🎯 Re-rankeando 80 chunks com Cross-Encoder...'
'⚡ Calculando scores de relevância...'


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

'⚠️ Nenhum chunk passou do threshold 0.2'
'📋 Usando 3 melhores chunks mesmo abaixo do threshold'
'✅ 3 chunks finais selecionados'
'📊 Scores finais: -4.317 (melhor) → -5.254 (pior)'
'\n📝 Etapa 4: Geração da resposta'
'📝 Gerando resposta final usando 3 chunks como contexto...'
'🤖 Enviando para LLM (contexto: 3412 caracteres)...'
'✅ Resposta gerada com sucesso!'
'\n============================================================'
'✅ PROCESSO CONCLUÍDO COM SUCESSO!'
'============================================================'
'❓ Pergunta: A obra menciona alguma comida ou prato típico?'
'\n💡 Resposta: Não foi possível responder com base no contexto fornecido.'
'============================================================'
